## Circuit cutting with specified cut points using Circuit Knitting Toolbox

### Import relevant modules

In [1]:
import numpy as np
from qiskit import QuantumCircuit

from circuit_knitting_toolbox.circuit_cutting import WireCutter

### Create a circuit to cut

In [2]:
qc = QuantumCircuit(5)
for i in range(5):
    qc.h(i)
qc.cx(0, 1)
for i in range(2, 5):
    qc.t(i)
qc.cx(0, 2)
qc.rx(np.pi / 2, 4)
qc.rx(np.pi / 2, 0)
qc.rx(np.pi / 2, 1)
qc.cx(2, 4)
qc.t(0)
qc.t(1)
qc.cx(2, 3)
qc.ry(np.pi / 2, 4)
for i in range(5):
    qc.h(i)

qc.draw()

┌───┐                ┌─────────┐   ┌───┐   ┌───┐
q_0: ┤ H ├──■───────■─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤┌─┴─┐     │     ├─────────┤   ├───┤   ├───┤
q_1: ┤ H ├┤ X ├─────┼─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤├───┤   ┌─┴─┐   └─────────┘   └───┘   ├───┤
q_2: ┤ H ├┤ T ├───┤ X ├────────■──────────■─────┤ H ├
     ├───┤├───┤   └───┘        │        ┌─┴─┐   ├───┤
q_3: ┤ H ├┤ T ├────────────────┼────────┤ X ├───┤ H ├
     ├───┤├───┤┌─────────┐   ┌─┴─┐   ┌──┴───┴──┐├───┤
q_4: ┤ H ├┤ T ├┤ Rx(π/2) ├───┤ X ├───┤ Ry(π/2) ├┤ H ├
     └───┘└───┘└─────────┘   └───┘   └─────────┘└───┘

### Load a WireCutter object with our QuantumCircuit and cut the circuit at the specified location(s)

In [3]:
cutter = WireCutter(qc)
cuts = cutter.cut_manual(subcircuit_vertices=[[0, 1], [2, 3]])

--------------------
subcircuit 0
ρ qubits = 0, O qubits = 1, width = 3, effective = 2, depth = 6, size = 12
     ┌───┐                     ┌─────────┐┌───┐┌───┐
q_0: ┤ H ├──■───────────────■──┤ Rx(π/2) ├┤ T ├┤ H ├
     ├───┤┌─┴─┐┌─────────┐  │  └──┬───┬──┘├───┤└───┘
q_1: ┤ H ├┤ X ├┤ Rx(π/2) ├──┼─────┤ T ├───┤ H ├─────
     ├───┤├───┤└─────────┘┌─┴─┐   └───┘   └───┘     
q_2: ┤ H ├┤ T ├───────────┤ X ├─────────────────────
     └───┘└───┘           └───┘                     
subcircuit 1
ρ qubits = 1, O qubits = 0, width = 3, effective = 3, depth = 6, size = 11
                                          ┌───┐
q_0: ───────────────────────■───────■─────┤ H ├
     ┌───┐┌───┐             │     ┌─┴─┐   ├───┤
q_1: ┤ H ├┤ T ├─────────────┼─────┤ X ├───┤ H ├
     ├───┤├───┤┌─────────┐┌─┴─┐┌──┴───┴──┐├───┤
q_2: ┤ H ├┤ T ├┤ Rx(π/2) ├┤ X ├┤ Ry(π/2) ├┤ H ├
     └───┘└───┘└─────────┘└───┘└─────────┘└───┘
Estimated cost = 1.280e+02
--------------------


### Define a constant shots function, evaluate our circuit, and verify the results

In [21]:
def num_shots_fn(circuit):
    return 1024


subcircuit_instance_probabilities = cutter.evaluate(cuts, mode="sv", num_shots_fn=num_shots_fn)
ordered_probabilities = cutter.recompose(subcircuit_instance_probs, cuts, num_threads=4)
metrics = cutter.verify(ordered_probabilities)

print(metrics)

{'nearest': {'chi2': 0, 'Mean Squared Error': 3.752148580465971e-34, 'Mean Absolute Percentage Error': 7.760053442355182e-14, 'Cross Entropy': 2.599681088367842, 'HOP': 0.9004283905932733}, 'naive': {'chi2': 0, 'Mean Squared Error': 2.793021597327788e-33, 'Mean Absolute Percentage Error': 7.625290711954506e-14, 'Cross Entropy': 2.5996810883678414, 'HOP': 0.900428390593274}}
